In [ ]:
import boto3
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
bucket = session.default_bucket()

In [ ]:
prefix = 'dogscats'
s3_train_path = 's3://{}/{}/input/train/'.format(bucket, prefix)
s3_val_path   = 's3://{}/{}/input/validation/'.format(bucket, prefix)
s3_output     = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

### Get the name of the image classification algorithm in our region

In [ ]:
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('image-classification', region)
print(container)

### Configure the training job

In [ ]:
role = sagemaker.get_execution_role()

ic = sagemaker.estimator.Estimator(container,
                                   role, 
                                   instance_count=1, 
                                   instance_type='ml.p3.2xlarge',
                                   output_path=s3_output)

### Set algorithm parameters

In [ ]:
#precision_dtype = 'float16'
precision_dtype = 'float32'

ic.set_hyperparameters(num_layers=50,               # Train a Resnet-50 model
                       use_pretrained_model=0,      # Train from scratch
                       num_classes=2,               # Dogs and cats
                       num_training_samples=22500,  # Number of training samples
                       mini_batch_size=128,
                       precision_dtype=precision_dtype,
                       epochs=10)                   # Learn the training samples 30 times

### Set dataset parameters

In [ ]:
train_data = sagemaker.TrainingInput(s3_train_path, 
                                        distribution='FullyReplicated', 
                                        content_type='application/x-recordio',
                                        s3_data_type='S3Prefix')

validation_data = sagemaker.TrainingInput(s3_val_path,
                                             distribution='FullyReplicated', 
                                             content_type='application/x-recordio', 
                                             s3_data_type='S3Prefix')

s3_channels = {'train': train_data, 'validation': validation_data}

### Train the model

In [ ]:
ic.fit(inputs=s3_channels)

### Deploy the model

In [ ]:
import time

ic_endpoint_name = 'ic-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

ic_predictor = ic.deploy(initial_instance_count=1,
                         instance_type='ml.c5.4xlarge',
                         endpoint_name=ic_endpoint_name,
                         wait=False)

## Compile and deploy the model with Neo

In [ ]:
output_path = 's3://{}/{}/output-neo/'.format(bucket, prefix)

ic_neo_model = ic.compile_model(target_instance_family='ml_c5', 
                                   input_shape={'data':[1, 3, 224, 224]},
                                   role=role,
                                   framework='mxnet',
                                   framework_version='1.5.1',
                                   output_path=output_path)

In [ ]:
ic_neo_endpoint_name = 'ic-neo-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(ic_neo_endpoint_name)

ic_neo_model.image_uri = image_uris.retrieve('image-classification-neo', region)
print(ic_neo_model.image_uri)


ic_neo_predictor = ic_neo_model.deploy(endpoint_name=ic_neo_endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.c5.4xlarge')

### Download a test image

In [ ]:
!wget -O /tmp/test.jpg https://upload.wikimedia.org/wikipedia/commons/b/b7/LabradorWeaving.jpg
file_name = '/tmp/test.jpg'
from IPython.display import Image
Image(file_name)

### Predict test image

In [ ]:
# Load test image from file
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

def predict_images(predictor, iterations=1000):
    total = 0
    predictor.content_type = 'application/x-image'
    for i in range(0, iterations):
        tick = time.time()
        response = predictor.predict(payload)
        tock = time.time()
        total += tock-tick
    return total/iterations

In [ ]:
%%time
predict_images(ic_predictor)

In [ ]:
%%time
predict_images(ic_neo_predictor)

In [ ]:
%%sh -s $output_path
echo $1
aws s3 ls $1
aws s3 cp $1model-ml_c5.tar.gz .
tar xvfz model-ml_c5.tar.gz

### Delete endpoints

In [ ]:
ic_predictor.delete_endpoint()

In [ ]:
ic_neo_predictor.delete_endpoint()

In [ ]:
output_path = 's3://{}/{}/output-neo/'.format(bucket, prefix)

ic_neo_model = ic.compile_model(target_instance_family='rasp3b', 
                                   input_shape={'data':[1, 3, 224, 224]},
                                   role=role,
                                   framework='mxnet',
                                   framework_version='1.5.1',
                                   output_path=output_path)